## importing mediapipe and open cv libraries

In [3]:
import mediapipe as mp
import cv2

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

thres = 0.5

classNames= []
classFile = "coco.names"
with open(classFile,"rt") as f:
    classNames = f.read().rstrip("n").split("n")

configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

# using detection model 
net = cv2.dnn_DetectionModel(weightsPath,configPath) 
# default config I found on the documentation 
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

< cv2.dnn.Model 0x7fce31b06cb0>

# WEBCAM INPUT

## writing the coordinate data to a file

In [5]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
    classIds, confs, bbox = net.detect(image,confThreshold=thres)
    print(classIds,bbox)
 
    if len(classIds) != 0:
      for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
        cv2.rectangle(image,box,color=(0,255,0),thickness=5)
        cv2.putText(image,classNames[classId-1].upper(),(box[0]+10,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.putText(image,str(round(confidence*100,2)),(box[0]+200,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

# Closing all the frames and windows 
cv2.waitKey(0)
cv2.destroyAllWindows() 
cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


() ()
[1] [[ 677  412 1060  663]]
[1] [[ 569  408 1192  671]]
[1] [[ 606  409 1148  671]]
[ 1 73] [[ 580  407 1184  673]
 [  14  526  914  554]]


IndexError: list index out of range

In [ ]:
if type(results.multi_hand_world_landmarks) == list: 
  for landmarks in results.multi_hand_world_landmarks:
    x = [round(abs(landmark.x)*100,2) for landmark in landmarks.landmark]
    y = [round(abs(landmark.y)*100,2) for landmark in landmarks.landmark]
    z = [round(abs(landmark.z)*100,2) for landmark in landmarks.landmark]
    print("x coordinate")
    print(x) 
    print("y coordinate")
    print(y)
    print("z coordinate")
    print(z)

In [ ]:
# printing the average value of the 21 coordinates 
avg_x = sum(x)/len(x)
print(f"average value of the x coordinates: {avg_x: .2f}cm")

avg_y = sum(y)/len(y)
print(f"average value of the y coordinates: {avg_y: .2f}cm")

avg_z = sum(z)/len(z)
print(f"average value of the z coordinates: {avg_z: .2f}cm")